In [1]:
import datetime
import mysql.connector
import config


def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

db = init_db()
cursor = get_cursor(db)

In [2]:
from nltk.tokenize import sent_tokenize
import demoji
import unicodedata as uni
import pandas as pd

def handle_emoji(text):
    emoji_less_text = demoji.replace(text, "")

    return emoji_less_text

def handle_unicode(text):
    text = uni.normalize('NFKD', text)

    return text

def case_folding(text):
    return text.lower()

def escape_char_clean(text):
    return text.replace('\n', ' ')

sql_query = """
SELECT text
FROM hotel_review_filtered
WHERE lang=%s
"""
sql_args = ('in',)

cursor.execute(sql_query, sql_args)
ori_data = cursor.fetchall()

num_of_ori = len(ori_data)

elapsed = 0
num_of_sentence = 0
processed_sentences = list()
for data in ori_data:
    list_of_sentence = sent_tokenize(data[0])
    for sentence in list_of_sentence:
        num_of_words = len([word for word in sentence.split()])
        if num_of_words > 2:
            sentence = handle_emoji(sentence)
            sentence = handle_unicode(sentence)
            sentence = case_folding(sentence)
            sentence = escape_char_clean(sentence)
            processed_sentences.append(sentence)
            num_of_sentence += 1
    elapsed += 1
    print(f'\rSentences: {num_of_sentence:8} | Elapsed: {elapsed:8}/{num_of_ori}', end='')

df = pd.DataFrame(processed_sentences, columns=['text'])
print(len(df))
df.to_csv(f'./data/allsentence/{sql_args[0]}_allsentence.csv', index=False, header=False)

Sentences:   248809 | Elapsed:   100350/100350248809


In [18]:
sql_args = ('in',)

In [19]:
# create csv dataset for training (250 each)
import pandas as pd
import random

df = pd.read_csv(r'C:\Users\MSI-GAMMING\Documents\COOLYEAH\.skripsi\Code\DataProcessing\data\allsentence\in_allsentence.csv',
                 header=None)
all_sentence = df[0].values.tolist()

size = len(all_sentence)
cut = int(0.05*size)
shuffled_data = all_sentence.copy()
random.shuffle(shuffled_data)

for_model = shuffled_data[:cut]
leftover = shuffled_data[cut:]

pd.DataFrame(leftover).to_csv(f'.\\data\\leftover\\{sql_args[0]}_leftoversentence.csv', index=False, header=False)

split_size = 250
doc_num = 0
_ = 0
batch_list = list()
for sentence in for_model:
    batch_list.append(sentence)
    _ += 1
    if _ == split_size:
        pd.DataFrame(batch_list).to_csv(f'.\\data\\formodel\\{sql_args[0]}_formodel_{doc_num}_{len(batch_list)}.csv', index=False, header=False)
        batch_list = list()
        _ = 0
        doc_num += 1
pd.DataFrame(batch_list).to_csv(f'.\\data\\formodel\\{sql_args[0]}_formodel_{doc_num}_{len(batch_list)}.csv', index=False, header=False)


In [12]:
print(shuffled_data)

None
